In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import re

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from collections import defaultdict
from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split

MODEL_PRETRAIN = 'indobenchmark/indobert-base-p2'

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PRETRAIN,  truncation=True)

In [4]:
DEBUG = False
TARGET = 'POI/street'
POI = 'POI'
STREET = 'street'
SEED = 2021

In [6]:
import pickle
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/BERT/bert-base-predictions.pkl', 'rb') as f:
    ALLBERT = pickle.load(f)

In [7]:
df_tmp = pd.read_parquet('C:/Users/Ale/Desktop/jupyter_folder/FYP/BERT/df.pqt')

In [8]:
df_tmp['allbert'] = ALLBERT

In [9]:
train = pd.read_parquet('C:/Users/Ale/Desktop/jupyter_folder/FYP/train.pqt')

In [10]:
val = pd.read_parquet('C:/Users/Ale/Desktop/jupyter_folder/FYP/val.pqt')

In [11]:
test = pd.read_parquet('C:/Users/Ale/Desktop/jupyter_folder/FYP/test.pqt')

In [12]:
import pickle
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/train_predicts.pkl', 'rb') as f:
    train_predicts = pickle.load(f)
import pickle
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

In [13]:
import numpy as np

def reconstruct_tokens(sent, prediction):
    sent = sent[1:-1]
    prediction = prediction[1:-1]
    
    sentences = []
    predictions = []

    last_token = ""
    las_predict = []

    for idx, token in enumerate(sent):
        if token.startswith("##"):
            predictions[-1].append(prediction[idx])
            sentences[-1] += token.lstrip("##")
        else:
            sentences.append(token)
            predictions.append([prediction[idx]])
    return sentences, [np.mean(p, axis=0) for p in predictions]

In [14]:
current_predict = train_predicts[0][0]

In [15]:
current_mask = train_data[0]['attention_mask'][0]

In [16]:
current_token = tokenizer.convert_ids_to_tokens(tokenizer(train.iloc[0]['clean_tokens'].tolist(), truncation=True, padding=True, is_split_into_words=True)['input_ids'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
fin_token, fin_predict = reconstruct_tokens(current_token, [x for i,x in enumerate(current_predict) if current_mask[i]])

In [18]:
assert len(fin_token) == len(fin_predict)

def reconstruct_all(predictions, data_pkl, data_df):
    all_tokens = []
    all_predicts = []
    flat_idx = 0
    for idx, pred in tqdm(enumerate(predictions), total=len(predictions)):
        for iidx, current_predict in enumerate(pred):
            current_mask = data_pkl[idx]['attention_mask'][iidx]
            current_token = tokenizer.convert_ids_to_tokens(
                tokenizer(data_df.iloc[flat_idx]['clean_tokens'].tolist(), truncation=True, padding=True, is_split_into_words=True)['input_ids']
            )
            fin_token, fin_predict = reconstruct_tokens(current_token, [x for i,x in enumerate(current_predict) if current_mask[i]])
            assert len(fin_token) == len(fin_predict)
            
            all_tokens.append(fin_token)
            all_predicts.append(fin_predict)
            flat_idx += 1
    return all_tokens, all_predicts

In [19]:
all_train_tokens, all_train_predict = reconstruct_all(train_predicts, train_data, train)

100%|██████████| 7024/7024 [03:01<00:00, 38.75it/s]


In [20]:
train['tokens'] = all_train_tokens
train['predict'] = all_train_predict

In [21]:
import gc
del(train_predicts, train_data)
gc.collect()

68

In [22]:
train = train.set_index('id').join(df_tmp[df_tmp['split'] != 'test'].set_index('id')['allbert'], how='left')

In [23]:
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/val_predicts.pkl', 'rb') as f:
    val_predicts = pickle.load(f)

with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/val_data.pkl', 'rb') as f:
    val_data = pickle.load(f)

In [24]:
all_val_tokens, all_val_predict = reconstruct_all(val_predicts, val_data, val)

100%|██████████| 2342/2342 [01:00<00:00, 39.00it/s]


In [25]:
val['tokens'] = all_val_tokens
val['predict'] = all_val_predict

In [26]:
del(val_predicts, val_data)
gc.collect()

68

In [27]:
val = val.set_index('id').join(df_tmp[df_tmp['split'] != 'test'].set_index('id')['allbert'], how='left')

In [28]:
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/test_predicts.pkl', 'rb') as f:
    test_predicts = pickle.load(f)

with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [29]:
all_test_tokens, all_test_predict = reconstruct_all(test_predicts, test_data, test)

100%|██████████| 3125/3125 [00:39<00:00, 79.23it/s]


In [30]:
test['tokens'] = all_test_tokens
test['predict'] = all_test_predict

In [31]:
del(test_predicts, test_data)
gc.collect()

68

In [32]:
test = test.set_index('id').join(df_tmp[df_tmp['split'] == 'test'].set_index('id')['allbert'], how='left')

In [33]:
test_allbert_len = test['allbert'].apply(len)
test_token_len = test['tokens'].apply(len)
test[test_token_len == test_allbert_len]

raw_address  \
id                                                         
0                  s. par 53 sidanegara 4 cilacap tengah   
1                  angg per, baloi indah kel. lubuk baja   
2                                  asma laun, mand imog,   
3          ud agung rej, raya nga sri wedari karanganyar   
4                             cut mutia, 35 baiturrahman   
...                                                  ...   
49995                    toko mbak farid semboro semboro   
49996     vie - tk. ridho kids, vete 3 cari, 16720 ciawi   
49997                mart dan roti bakar malabar, nasio,   
49998  graha indah pamulang jl. mujair raya bambu apu...   
49999                                               adi,   

                                            clean_tokens  \
id                                                         
0        [s, ., par, 53, sidanegara, 4, cilacap, tengah]   
1      [angg, per, ,, baloi, indah, kel, ., lubuk, baja]   
2                         [asma, laun, ,, mand, imog, ,]   
3      [ud, agung, rej, ,, raya, nga, sri, wedari, ka...   
4                      [cut, mutia, ,, 35, baiturrahman]   
...                                                  ...   
49995              [toko, mbak, farid, semboro, semboro]   
49996  [vie, -, tk, ., ridho, kids, ,, vete, 3, cari,...   
49997     [mart, dan, roti, bakar, malabar, ,, nasio, ,]   
49998  [graha, indah, pamulang, jl, ., mujair, raya, ...   
49999                                           [adi, ,]   

                                                  tokens  \
id                                                         
0        [s, ., par, 53, sidanegara, 4, cilacap, tengah]   
1      [angg, per, ,, baloi, indah, kel, ., lubuk, baja]   
2                         [asma, laun, ,, mand, imog, ,]   
3      [ud, agung, rej, ,, raya, nga, sri, wedari, ka...   
4                      [cut, mutia, ,, 35, baiturrahman]   
...                                                  ...   
49995              [toko, mbak, farid, semboro, semboro]   
49996  [vie, -, tk, ., ridho, kids, ,, vete, 3, cari,...   
49997     [mart, dan, roti, bakar, malabar, ,, nasio, ,]   
49998  [graha, indah, pamulang, jl, ., mujair, raya, ...   
49999                                           [adi, ,]   

                                                 predict  \
id                                                         
0      [[0.0924, 0.907, 0.00010127, 8.494e-05, 8.994e...   
1      [[0.09454, 0.9053, 8.833e-05, 0.00011665, 0.00...   
2      [[0.08417, 0.0001241, 0.00011826, 0.9155, 0.00...   
3      [[0.2126, 0.0007358, 0.0001147, 0.7866, 5.704e...   
4      [[0.1415, 0.8584, 5.716e-05, 0.0001322, 9.4e-0...   
...                                                  ...   
49995  [[0.11884, 0.000168, 0.0001057, 0.881, 6.324e-...   
49996  [[0.01913, 4.95e-05, 8.464e-05, 0.9785, 0.0021...   
49997  [[0.08844, 0.0001366, 0.0001016, 0.911, 0.0001...   
49998  [[0.0874, 0.0001007, 9.39e-05, 0.912, 0.000310...   
49999  [[0.1866, 0.594, 0.0001272, 0.2189, 0.0001572]...   

                                                 allbert  
id                                                        
0      [[0.0004358, 0.0004532, 0.001666, 0.0005536, 0...  
1      [[0.001118, 0.0005226, 0.001705, 0.0009313, 0....  
2      [[0.0001712, 0.0003023, 0.0009313, 0.0003448, ...  
3      [[0.00142, 0.0004778, 0.002388, 0.0115, 0.0081...  
4      [[9.12e-05, 3.254e-05, 0.000189, 0.0001005, 8....  
...                                                  ...  
49995  [[0.0003917, 0.0001664, 0.002323, 0.01572, 0.0...  
49996  [[0.0004478, 0.00012165, 0.00106, 0.0002286, 0...  
49997  [[0.0005407, 0.00011665, 0.002573, 0.00177, 0....  
49998  [[6.02e-05, 1.305e-05, 0.001488, 0.003796, 0.0...  
49999  [[0.0001676, 5.25e-05, 2.354e-05, 1.66e-05, 5....  

[50000 rows x 5 columns]

In [34]:
test.head(2)

raw_address  \
id                                          
0   s. par 53 sidanegara 4 cilacap tengah   
1   angg per, baloi indah kel. lubuk baja   

                                         clean_tokens  \
id                                                      
0     [s, ., par, 53, sidanegara, 4, cilacap, tengah]   
1   [angg, per, ,, baloi, indah, kel, ., lubuk, baja]   

                                               tokens  \
id                                                      
0     [s, ., par, 53, sidanegara, 4, cilacap, tengah]   
1   [angg, per, ,, baloi, indah, kel, ., lubuk, baja]   

                                              predict  \
id                                                      
0   [[0.0924, 0.907, 0.00010127, 8.494e-05, 8.994e...   
1   [[0.09454, 0.9053, 8.833e-05, 0.00011665, 0.00...   

                                              allbert  
id                                                     
0   [[0.0004358, 0.0004532, 0.001666, 0.0005536, 0...  
1   [[0.001118, 0.0005226, 0.001705, 0.0009313, 0....

In [35]:
train.head(2)

raw_address  \
id                                                          
248525  richeese factory ramayana karawang. jl tuparev...   
240526  bc hni ketapang 1,jln. gajah mada dalam, arah ...   

                                               POI/street  \
id                                                          
248525                       ramayana karawang/jl tuparev   
240526  arah pesantren hidayatullah/jln. gajah mada dalam   

                                             clean_tokens  \
id                                                          
248525  [richeese, factory, ramayana, karawang, ., jl,...   
240526  [bc, hni, ketapang, 1, ,, jln, ., gajah, mada,...   

                                    POI                        street  \
id                                                                      
248525             [ramayana, karawang]                 [jl, tuparev]   
240526  [arah, pesantren, hidayatullah]  [jln, ., gajah, mada, dalam]   

                                                   labels  \
id                                                          
248525  [O, O, B-POI, I-POI, O, B-STREET, I-STREET, O,...   
240526  [O, O, O, O, O, B-STREET, I-STREET, I-STREET, ...   

                                                   tokens  \
id                                                          
248525  [richeese, factory, ramayana, karawang, ., jl,...   
240526  [bc, hni, ketapang, 1, ,, jln, ., gajah, mada,...   

                                                  predict  \
id                                                          
248525  [[1.0, 4.7e-05, 2.885e-05, 2.956e-05, 4.7e-06]...   
240526  [[1.0, 5.966e-05, 2.79e-05, 0.0001297, 4.95e-0...   

                                                  allbert  
id                                                         
248525  [[0.001473, 0.0002205, 0.005627, 0.0016165, 0....  
240526  [[0.000692, 0.0003912, 0.02365, 0.01816, 0.003...

In [36]:
val.head(2)

raw_address        POI/street  \
id                                                             
99792     kemu 44 ketabang rt 9 10 genteng             /kemu   
24401  med utara xxv b medokan ayu rungkut  /med utara xxv b   

                                      clean_tokens POI                street  \
id                                                                             
99792     [kemu, 44, ketabang, rt, 9, 10, genteng]  []                [kemu]   
24401  [med, utara, xxv, b, medokan, ayu, rungkut]  []  [med, utara, xxv, b]   

                                                  labels  \
id                                                         
99792                       [B-STREET, O, O, O, O, O, O]   
24401  [B-STREET, I-STREET, I-STREET, I-STREET, O, O, O]   

                                            tokens  \
id                                                   
99792     [kemu, 44, ketabang, rt, 9, 10, genteng]   
24401  [med, utara, xxv, b, medokan, ayu, rungkut]   

                                                 predict  \
id                                                         
99792  [[0.10913, 0.8906, 0.0001805, 0.00010717, 0.00...   
24401  [[0.0949, 0.905, 0.0001363, 0.0001196, 0.00011...   

                                                 allbert  
id                                                        
99792  [[0.000266, 0.0001225, 0.00035, 0.001495, 0.02...  
24401  [[0.002117, 0.00509, 0.0585, 0.00609, 0.06714,...

In [37]:
def get_len(x):
    try:
        return len(x)
    except:
        return 0

In [38]:
def naive_tokenize(x):
    if type(x) == str:
        return re.split('(\W)', x)
    return x

In [39]:
def tokenize(x):
    if type(x) == str:
        return tokenizer.convert_ids_to_tokens(tokenizer.encode(x))[1:-1]
    return x

In [40]:
def reconstruct_sent(sent):
    if type(sent) != list:
        return sent
    sentences = []
    for idx, token in enumerate(sent):
        if token.startswith("##"):
            sentences[-1] += token.lstrip("##")
        else:
            sentences.append(token)
    return sentences

In [41]:
def remap_UNK(x):
    return x.replace('[UNK]', "'")

def remap_UNK_list(X):
    try:
        return[remap_UNK(x) for x in X]
    except:
        X

In [42]:
train['tokens'] = train['tokens'].progress_apply(remap_UNK_list)

100%|██████████| 224740/224740 [00:00<00:00, 232787.63it/s]


In [43]:
val['tokens'] = val['tokens'].progress_apply(remap_UNK_list)

100%|██████████| 74914/74914 [00:00<00:00, 225959.69it/s]


In [44]:
test['tokens'] = test['tokens'].progress_apply(remap_UNK_list)

100%|██████████| 50000/50000 [00:01<00:00, 47379.44it/s]


In [45]:
import re
def is_roman_numeral(txt): 
    return bool(
        re.search(
            r"^M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$",\
            txt.upper()
        )
    )

In [46]:
def convert_token(token):
    if is_roman_numeral(token):
        return "<ROMAN_NUMERAL>"
    elif token.isdigit():
        return "<DIGIT>"
    elif token.isalpha():
        return token
    else:
        _result = set()
        for char in token:
            if char.isalpha():
                _result.add('ALPHA')
            elif char.isdigit():
                _result.add('DIGIT')
        return "<" + "_".join(sorted(list(_result))) + ">"

# def generate_pair(X):
#     return [(x, convert_token(x)) for x in X]

In [47]:
def word2features(sent, tags, allberts,  i):
    word = sent[i]
    tag = list(reversed(np.argsort(tags[i])))
    allbert = list(reversed(np.argsort(allberts[i])))

    
    features = {
        'tag-argmax': 'v{}'.format(str(tag[0])),
        'bert-argmax': 'v{}'.format(str(allbert[0])),
        'bias': 1.0,
        'pos': i,
        '-pos': len(sent) - i,
        'word.ishyphen': sent[i].strip('#') == '-',
        'word.isdot': sent[i].strip('#') == '.',
        'word.iscomma': sent[i].strip('#') == ',',
#         'word.isupper()': word.isupper(),
#         'word.isalpha()': word.isalpha(),        
#         'word.istitle()': word.istitle(),
#         'word.isdigit()': word.isdigit(),
#         'word.isroman': is_roman_numeral(word),
#         'tag': get_max(tag),
    }
    
    for topi in range(min(2, len(allbert))):
        vi = allbert[topi]
        vect = allberts[i][vi]
        features[f'albrt{vi}'] = vect

    for topi in range(min(8, len(tag))):
        vi = tag[topi]
        vect = tags[i][vi]
        features[f'v{vi}'] = vect
    
    word_strip = word.strip('#')
    features[':word.lower()'] = convert_token(word_strip.lower())
    if word_strip.isalpha():
        if len(word) >= 2:
            features[':word[-2:]'] = word_strip[-2:]                
            features[':word[:2]'] = word_strip[:2]
        if len(word) >= 3:
            features[':word[-3:]'] = word_strip[-3:]
            features[':word[:3]'] = word_strip[:3]
    
    if i > 0:
        word1 = sent[i-1]
        tag1 = list(reversed(np.argsort(tags[i-1])))

        features.update({
#             '-1:word.istitle()': word1.istitle(),
#             '-1:word.isupper()': word1.isupper(),
#             '-1:word.isdigit()': word1.isdigit(),
#             '-1:word.isroman': is_roman_numeral(word1),
            '-1:word.ishyphen': word1.strip('#') == '-',
            '-1:word.isdot': word1.strip('#') == '.',
            '-1:word.iscomma': word1.strip('#') == ',',          
            '-1:tag': tag1[0],
        })
        
        word1_strip = word1.strip('#')
        features['-1:word.lower()'] = convert_token(word1_strip.lower())
        if word1_strip.isalpha():            
            if len(word1_strip) >= 2:
                features['-1:word[-2:]'] = word1_strip[-2:]                    
                features['-1:word[:2]'] = word1_strip[:2]                    
            if len(word1_strip) >= 3:
                features['-1:word[-3:]'] = word1_strip[-3:]
                features['-1:word[:3]'] = word1_strip[:3]
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1]
        tag1 = list(reversed(np.argsort(tags[i+1])))
        features.update({
#             '+1:word.istitle()': word1.istitle(),
#             '+1:word.isupper()': word1.isupper(),
#             '+1:word.isdigit()': word1.isdigit(),
#             '+1:word.isroman': is_roman_numeral(word1),
            '+1:word.ishyphen': word1.strip('#') == '-',
            '+1:word.isdot': word1.strip('#') == '.',
            '+1:word.iscomma': word1.strip('#') == ',',           
            '+1:tag': tag1[0],
        })

        word1_strip = word1.strip('#')
        features['+1:word.lower()'] = convert_token(word1_strip.lower())
        if word1_strip.isalpha():
            if len(word1_strip) >= 2:
                features['+1:word[-2:]'] = word1_strip[-2:]
                features['+1:word[:2]'] = word1_strip[:2]                    
            if len(word1_strip) >= 3:
                features['+1:word[-3:]'] = word1_strip[-3:]
                features['+1:word[:3]'] = word1_strip[:3]
    else:
        features['EOS'] = True

    return features


def sent2features(sent, tags, allbert):
    return [word2features(sent, tags, allbert, i) for i in range(len(sent))]

In [48]:
train['sent'] = train[['tokens', 'predict', 'allbert']].progress_apply(lambda x: sent2features(*x), axis=1)

100%|██████████| 224740/224740 [01:59<00:00, 1877.24it/s]


In [49]:
train.iloc[0]['sent'][1]

{'tag-argmax': 'v0',
 'bert-argmax': 'v26',
 'bias': 1.0,
 'pos': 1,
 '-pos': 15,
 'word.ishyphen': False,
 'word.isdot': False,
 'word.iscomma': False,
 'albrt26': 0.5806,
 'albrt33': 0.183,
 'v0': 1.0,
 'v3': 6.765e-05,
 'v2': 3.81e-05,
 'v1': 3.49e-05,
 'v4': 2e-05,
 ':word.lower()': 'factory',
 ':word[-2:]': 'ry',
 ':word[:2]': 'fa',
 ':word[-3:]': 'ory',
 ':word[:3]': 'fac',
 '-1:word.ishyphen': False,
 '-1:word.isdot': False,
 '-1:word.iscomma': False,
 '-1:tag': 0,
 '-1:word.lower()': 'richeese',
 '-1:word[-2:]': 'se',
 '-1:word[:2]': 'ri',
 '-1:word[-3:]': 'ese',
 '-1:word[:3]': 'ric',
 '+1:word.ishyphen': False,
 '+1:word.isdot': False,
 '+1:word.iscomma': False,
 '+1:tag': 3,
 '+1:word.lower()': 'ramayana',
 '+1:word[-2:]': 'na',
 '+1:word[:2]': 'ra',
 '+1:word[-3:]': 'ana',
 '+1:word[:3]': 'ram'}

In [50]:
val['sent'] = val[['tokens', 'predict', 'allbert']].progress_apply(lambda x: sent2features(*x), axis=1)

100%|██████████| 74914/74914 [00:40<00:00, 1869.41it/s]


In [51]:
X_train, X_val, y_train, y_val = train['sent'], val['sent'],  train['labels'], val['labels']

In [52]:
!pip3 install -q sklearn-crfsuite

In [53]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.03,
          c2=0.03,
          max_iterations=500,
          all_possible_transitions=False,
         verbose=True)

In [54]:
try:
    crf.fit(X_train, y_train, X_val, y_val)
except AttributeError:
    pass

loading dev data to CRFsuite:   0%|          | 228/74914 [00:00<00:32, 2277.23it/s]

loading dev data to CRFsuite: 100%|██████████| 74914/74914 [00:32<00:00, 2300.40it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 393104
Seconds required: 11.875

L-BFGS optimization
c1: 0.030000
c2: 0.030000
num_memories: 6
max_iterations: 500
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=9.76  loss=2459071.38 active=385627 precision=0.124  recall=0.200  F1=0.153  Acc(item/seq)=0.618 0.108  feature_norm=0.50
Iter 2   time=3.79  loss=1930935.84 active=360258 precision=0.124  recall=0.200  F1=0.153  Acc(item/seq)=0.618 0.108  feature_norm=0.39
Iter 3   time=3.73  loss=1761007.64 active=369618 precision=0.505  recall=0.290  F1=0.288  Acc(item/seq)=0.633 0.010  feature_norm=0.41
Iter 4   time=3.74  loss=1679423.84 active=386394 precision=0.491  recall=0.297  F1=0.306  Acc(item/seq)=0.653 0.030  feature_norm=0.47
Iter 5   time=3.76  loss=1385417.68

In [55]:
pred_val = crf.predict(X_val)

In [56]:
len(pred_val)

74914

In [57]:
def get_maping(maping, token):
    if token not in maping:
        return token
    try:
        return max(maping[token], key=maping[token].get)
    except:
        print(token)
        return token

In [58]:
OTHER = 'O'
def generate_prediction(predicted, raw_address_tokens, use_maping=False, maping_poi=None, maping_street=None):
    _poi = []
    _street = []
    
    for idx, token in enumerate(raw_address_tokens):
        if predicted[idx] != OTHER:
            tag = predicted[idx].split('-')[1]
            _token = token
            if tag == 'POI':
                _poi.append(_token)
            else:
                _street.append(_token)
    _poi = tokenizer.convert_tokens_to_string(_poi)
    _street = tokenizer.convert_tokens_to_string(_street)
    if use_maping:
        _poi = ''.join([get_maping(maping_poi, token) for token in naive_tokenize(_poi)])
        _street = ''.join([get_maping(maping_street, token) for token in naive_tokenize(_street)])
    return  _poi+ '/' + _street

In [59]:
# val[val['POI/street'] == '/']

In [60]:
val['predict1']  = [generate_prediction(pred_val[i], val['tokens'].iloc[i]) for i in tqdm(range(len(pred_val)), total=len(pred_val))]


100%|██████████| 74914/74914 [00:01<00:00, 40953.12it/s]


In [61]:
(val[TARGET] == val['predict1']).mean()

0.5966041060415943

In [62]:
val['pred_list'] = pred_val
val['y_list'] = y_val

In [63]:
(val['pred_list'].apply(lambda x: 'x'.join(x)) == val['y_list'].apply(lambda x: 'x'.join(x))).mean()

0.7562271404543877

In [64]:
pred_train = crf.predict(X_train)

In [65]:
train['pred_list'] = pred_train
train['y_list'] = y_train

In [66]:
(train['pred_list'].apply(lambda x: 'x'.join(x)) == train['y_list'].apply(lambda x: 'x'.join(x))).mean()

0.9042449052238142

In [67]:
train['predict1']  = [generate_prediction(pred_train[i], train['tokens'].iloc[i]) for i in tqdm(range(len(pred_train)), total=len(pred_train))]

100%|██████████| 224740/224740 [00:05<00:00, 38693.09it/s]


In [68]:
(train[TARGET] == train['predict1']).mean()

0.7035240722612797

In [69]:
test['sent'] = test[['tokens', 'predict', 'allbert']].progress_apply(lambda x: sent2features(*x), axis=1)

100%|██████████| 50000/50000 [00:27<00:00, 1799.29it/s]


In [70]:
test['pred_list'] = crf.predict(test['sent'])

In [71]:
import pickle

with open('train_sent.pkl','wb') as f:
    pickle.dump(train['sent'].tolist(), f)
    
with open('test_sent.pkl','wb') as f:
    pickle.dump(test['sent'].tolist(), f)

with open('val_sent.pkl','wb') as f:
    pickle.dump(val['sent'].tolist(), f)

In [72]:
test.drop(['predict',  'sent', 'allbert'], axis=1).to_parquet('test.pqt')

In [73]:
val.drop(['predict',  'sent', 'allbert'], axis=1).to_parquet('val.pqt')

In [74]:
train.drop(['predict',  'sent', 'allbert'], axis=1).to_parquet('train.pqt')

In [75]:
crf.state_features_

{('tag-argmax:v0', 'O'): 1.317318,
 ('tag-argmax:v0', 'B-POI'): 1.127151,
 ('tag-argmax:v0', 'I-POI'): -0.770551,
 ('tag-argmax:v0', 'B-STREET'): 4.857169,
 ('tag-argmax:v0', 'I-STREET'): 2.091105,
 ('bert-argmax:v26', 'O'): 0.049031,
 ('bert-argmax:v26', 'B-POI'): -0.15154,
 ('bert-argmax:v26', 'I-POI'): -0.040222,
 ('bert-argmax:v26', 'B-STREET'): 0.00167,
 ('bert-argmax:v26', 'I-STREET'): -0.046453,
 ('bias', 'O'): 0.58753,
 ('bias', 'B-POI'): -0.063105,
 ('bias', 'I-POI'): -0.522269,
 ('bias', 'B-STREET'): 0.102905,
 ('bias', 'I-STREET'): -0.130796,
 ('pos', 'O'): 0.119994,
 ('pos', 'B-POI'): 0.096635,
 ('pos', 'I-POI'): 0.137453,
 ('pos', 'B-STREET'): 0.053662,
 ('pos', 'I-STREET'): 0.157873,
 ('-pos', 'O'): -0.117189,
 ('-pos', 'B-POI'): -0.146788,
 ('-pos', 'I-POI'): -0.10508,
 ('-pos', 'B-STREET'): -0.189755,
 ('-pos', 'I-STREET'): -0.098509,
 ('word.ishyphen', 'O'): -0.019332,
 ('word.ishyphen', 'B-POI'): -0.575516,
 ('word.ishyphen', 'I-POI'): -0.298033,
 ('word.ishyphen', 'B

In [76]:
with open('crf.pkl','wb') as f:
#     pickle.dump(test['sent'].tolist(), f)
    pickle.dump(crf, f)